# Previsão de tarifa de táxi de Nova Iorque
A base de dados foi extraída desta competição do Kaggle: https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/overview. Abaixo a descrição:

Nesta competição [...] você tem a tarefa de prever o valor da tarifa (inclusive pedágios) para uma corrida de táxi em Nova Iorque, dado os locais de partida e destino. Embora você possa obter uma estimativa básica baseada apenas na distância entre os dois pontos, isso resultará em um RMSE de 5 a 8, dependendo do modelo utilizado [...]. Seu desafio é ter resultados melhores do que esses usando técnicas de aprendizagem de máquina!

São disponibilizados duas bases de dados:

train.csv - Características de entrada e valor da taxa, que corresponde a base de dados de treinamento (aproximadamente 55M de linhas).
test.csv - Características de entrada para a base de dados de teste (aproximadamente 10K linhas). O seu objetivo é prever a taxa para cada linha.

## Instalação e importação das bibliotecas

In [ ]:
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
  Using cached https://github.com/matplotlib/basemap/archive/master.zip
  Created wheel for basemap: filename=basemap-1.2.1-cp36-cp36m-linux_x86_64.whl size=121756177 sha256=68dc853f5bb7c123cead225ada1165eec5dc3b4e2065c4dfaba4632dd8fa4ce7
  Stored in directory: /tmp/pip-ephem-wheel-cache-0w2mzyl9/wheels/98/4a/fc/ce719b75d97e646645c225f3332b1b217536100314922e9572
Successfully built basemap


In [ ]:
import pandas as pd
import numpy as np
import random
import zipfile
from mpl_toolkits.basemap import Basemap
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from collections import Counter

## Montando drive e carregando  base de dados de treinamento

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#dataset_path = '/content/drive/My Drive/DataSet/new-york-city-taxi-fare-prediction.zip'
#zip_object = zipfile.ZipFile(file = dataset_path, mode = "r")
#zip_object.extractall('./')
#zip_object.close()

In [ ]:
#!head train.csv

In [ ]:
#number_rows_infile = sum(1 for line in open('train.csv'))

#number_rows_infile

In [ ]:
#sample_size = 250000

In [ ]:
#random.seed(24)

In [ ]:
#skip_rows = sorted(random.sample(range(1,number_rows_infile +1 ),number_rows_infile - sample_size))

In [ ]:
#len(skip_rows)

## Salvando a amostra em um arquivo csv

In [ ]:
#train = pd.read_csv('train.csv',skiprows= skip_rows)

In [ ]:
#train.shape

In [ ]:
#train.head()

In [ ]:
#train.to_csv('train_sample_' + str (sample_size) + '.csv', index= False)

## Analise de dados

In [ ]:
#train = pd.read_csv('train_sample_' + str(sample_size) + '.csv',
                   # parse_dates = ['key', 'pickup_datetime'],
                   # dtype = {'fare_amount': np.float32,
                            # 'pickup_latitude': np.float32,
                            # 'pickup_longitude': np.float32,
                            # 'dropoff_latitude': np.float32,
                            # 'dropoff_longitude': np.float32,
                            # 'passenger_count': np.int32             
                    })

Carregando dados 

In [ ]:
train = pd.read_csv('/content/drive/My Drive/DataSet/train_sample_250000.csv',
                    parse_dates = ['key', 'pickup_datetime'],
                    dtype = {'fare_amount': np.float32,
                             'pickup_latitude': np.float32,
                             'pickup_longitude': np.float32,
                             'dropoff_latitude': np.float32,
                             'dropoff_longitude': np.float32,
                             'passenger_count': np.int32             
                    })

Visualizando os dados formatação e informações básicas 

In [ ]:
train.head()

In [ ]:
train.info()

Não temos valores nulos para tatar 

In [ ]:
train.isna().sum()

Vamos verificar o formato da nossa coluna key

In [ ]:
key = train['key']

In [ ]:
key.describe()

Precisamos retirar os números depois dos segundo para os dados poderem ser trabalhados 

In [ ]:
key.head()

In [ ]:
plt.hist(key);

A coluna pickup datetime tem a mesma formatação da coluna key precisaremos também que remover os valores depois de segundos.

In [ ]:
datetime = train['pickup_datetime']

In [ ]:
datetime.describe()

Retirando as informacoes depois dos segundos 

In [ ]:
datetime = datetime.map(lambda date: date.tz_localize(None))

In [ ]:
datetime.head()

Deixando colunas key com o formato data igual datetime

In [ ]:
key = pd.to_datetime(key.dt.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
key.head()

Comparando coluna key e datetime para ver se são os duplicados e verificamos que são duplicados

In [ ]:
(key == datetime).value_counts()

Dropando a colunas Key duplicada

In [ ]:
train = train.drop(['key'],axis= 1)

Verificando o dataset e convertendo colunas pickup_datetime

In [ ]:
train.head()

In [ ]:
train['pickup_datetime'] = datetime

In [ ]:
train.head()

Vamos verificar a distribuicao dos dados de latitude e longitude  

In [ ]:
pik_long = train['pickup_longitude']

In [ ]:
pik_long.describe()

In [ ]:
sns.boxplot(pik_long,)

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(pik_long);

In [ ]:
pik_long_val =(pik_long > pik_long.median()-0.5 )& (pik_long < pik_long.median()+0.5)

In [ ]:
pik_long_val.value_counts()

É possível ver que nossos dados estão distribuidos entre -72 e -73 segundo os quartis  a mediana e distribuição nos gráficos , mas os valores máximo e mínimos estão bem fora da nossa distribuição

In [ ]:
pik_lat = train['pickup_latitude']

In [ ]:
pik_lat.describe()

In [ ]:
sns.boxplot(pik_lat)

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(pik_lat)

In [ ]:
pik_lat_val =(pik_lat > pik_lat.median()-0.5 )& (pik_lat < pik_lat.median()+0.5)

In [ ]:
pik_lat_val.value_counts()

É possível ver que nossos dados estão distribuidosentre 39 e 40 segundo os quartis e a mediana , mas os valores máximo e mínimos estão bem fora da nossa distribuição

In [ ]:
drop = train['dropoff_latitude']

In [ ]:
drop.describe()

In [ ]:
sns.boxplot(drop)

In [ ]:
sns.distplot(drop)

In [ ]:
drop_lat_val =(drop > drop.median()-0.5 )& (drop < drop.median()+0.5)

In [ ]:
drop_lat_val.value_counts()

É possível ver que nossos dados estão distribuidosentre 39 e 40 segundo os quartis e a mediana , mas os valores máximo e mínimos estão bem fora da nossa distribuição

In [ ]:
drop_long = train['dropoff_longitude']

In [ ]:
drop_long.describe()

In [ ]:
sns.boxplot(drop_long)

In [ ]:
sns.distplot(drop_long)

In [ ]:
drop_long_val =(drop_long> drop_long.median()-0.5 )& (drop_long < drop_long.median()+0.5)

In [ ]:
drop_long_val.value_counts()

É possível ver que nossos dados estão distribuidosentre -72 e -73 segundo os quartis e a mediana , mas os valores máximo e mínimos estão bem fora da nossa distribuição

Vamos analizar a coluna passenger_count

In [ ]:
passcount = train['passenger_count']

In [ ]:
passcount.describe()

Podemos ver que os dados se distribuem em torno de 1  a 2 passageiros que o mínimo e o máximo podem estar com valors errados pois com 0 passageiros não se faz corrida e com 208 passageiros é impossível em um táxi.

In [ ]:
sns.boxplot(passcount)

Podemos ver que temos menos de 1% dos valores da base de dados igual a zero então vamos optar por exclui los.

In [ ]:
print('Total',(passcount == 0).sum(),    'Porcentagem',(passcount == 0).mean())

In [ ]:
menos_20 = train.query('passenger_count <=20')
menos_20.head()

In [ ]:
plt.hist(menos_20['passenger_count'],bins=100);

Podemos ver que o número máximo de passageiros são 6 assim sendo valores maiores são outliers.

In [ ]:
passcount_val =((passcount>=1) & (passcount<=6))

passcount_val.value_counts()

Vamos verificar a coluna fare_amount

In [ ]:
fare = train['fare_amount']

In [ ]:
fare.describe()

In [ ]:
plt.hist(fare[fare > 50],bins= 100);

Conseguimos vizualizar que o valores estão distribuidos entre 6 a 12.5 dolares , que os valores máximos e mínimos podem ser outliers. No histograma conseguimos ver que temos valores até próximo de 150 dolares então vamos trabalhar com esse valor máximo.

In [ ]:
plt.hist(fare[fare<=12.5],bins= 100);

Fazendo um histograma para vermos somente os valores dentro da distribuição até 12.5 dolares conseguimos ver que temos valores de pouco abaixo de 2 dolares  a 12.5 dolares, vamos trabalhar com o valor mínimo de 2 dolares por ser taxi dificilmente o valor mínimo da corrida seja menor que isso.

In [ ]:
fare_value = (fare >2) & (fare<150)

print(fare_value.value_counts(), fare_value.value_counts(normalize=True))


Podemos ver que os valores fora da faixa estabelecida de 2 a 150 dolares são menos de 1 % da base de dados totalizando 37 valores de 250 000

Vamos concatenar todas as series que analizamos até o momento com os valores boleanos.

In [ ]:
valor_entr = pik_lat_val & pik_long_val & fare_value & passcount_val & drop_lat_val & drop_long_val

In [ ]:
print(valor_entr.value_counts(), '\n',valor_entr.value_counts(normalize= True))

Vamos dropar os valores outliers de todas as features que analisamos no dataset train

In [ ]:
train = train.drop(valor_entr[valor_entr== False].index)

In [ ]:
train.head()

In [ ]:
train.shape

# Feature Engeneering

Criando uma nova coluna apenas com a hora de embarque

In [ ]:
train['hour_of_day'] = train['pickup_datetime'].map(lambda date : date.timetuple().tm_hour)

In [ ]:
train.head()

Criando mais features, dia da semana , dia do ano e ano

In [ ]:
train['day_of_week'] = train['pickup_datetime'].map(lambda date : date.timetuple().tm_wday)
train['day_of_year'] = train['pickup_datetime'].map(lambda date : date.timetuple().tm_yday)
train['year'] = train['pickup_datetime'].map(lambda date : date.timetuple().tm_year)

In [ ]:
train.head()

Como estamos trabalhando com anos tem ano que é bi-sexto e não tem dia 29-02 temos que verificar , primeiro vamos verificar quantos registro temos nessa data.

In [ ]:
len(train['pickup_datetime'][train['pickup_datetime'].dt.strftime('%m-%d')=='02-29'])

Como temos apenas 117 registros de uma base de dados de quase 250 000 dados é mais prático apagar os registro do que verificar quais anos são bi-sextos

In [ ]:
train.drop(train['pickup_datetime'][train['pickup_datetime'].dt.strftime('%m-%d')=='02-29'].index ,inplace= True)

In [ ]:
train.shape

Como apagamos a data 29-02 do ano de 2012 que é bi-sexto a contagem de dias da coluna day_of_year vai estar errada em 1 dia podemos alterar feriados e data e interfir na nossa predição. Quando chegar no dia 28-2-2012 que é o dia 59 o próximo dia que seria 01-03 -2012 vai se tornar o novo 59.

In [ ]:
condicao = (train['year']== 2012) & (train['day_of_year'] >59)

train['day_of_year'] [condicao] = train['day_of_year'] -1

O ultimo ano da nossa base da dados é 2015 e podemos ver que não temos mais dados quando chegamos no dia 200 do ano então nossos dados de 2015 vai até metade do ano apenas.

In [ ]:
train['year'].unique()

In [ ]:
dados_2015 = train.query('day_of_year>180 and year == 2015')

dados_2015.count()

In [ ]:
dados_2015 = train.query('day_of_year>200 and year == 2015')

dados_2015.count()

Vamos criar algumas visualizações retirando o ano de 2015 que não temos dados do ano todo então estamos visualizando os anos de 2009 a 2014

In [ ]:
fig ,ax = plt.subplots(1,4 , figsize = (20,4))
train_not_2015 = train[train['year']<2015]
ax[0].hist(train_not_2015['hour_of_day'],bins = 24)
ax[0].set_title('Hour_of_day')
ax[1].hist(train_not_2015['day_of_week'],bins = 7)
ax[1].set_title('Day_of_week')
ax[2].hist(train_not_2015['day_of_year'],bins = 365)
ax[2].set_title('Day_of_year')
ax[3].hist(train_not_2015['year'],bins = 6)
ax[3].set_title('Year')

Podemos ver que temos variações significativas tanto na hora do dia que o fluxo é bem menor até as 5 da manhã , dia da semana que o dia 6 e 0 são os de menores que são respectivamente domingo e sabado e o dia do ano na metade do ano e no final do ano diminui os volume de corrida apenas entre anos não temos tanta variação.

In [ ]:
plt.hist(train['pickup_datetime'],bins=24);

Nesse scatter podemos ver que de 2009 até parte de 2012 temos praticamente uma linha de valores próximo de 50 o que poderia ser uma tarifa fixa para corridas tipo do aeroporto da o centro e vemos que a partir de 2013 essa linha sobre para próximo de 60 que poderia ser algum reajuste de valor 

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(x=train['pickup_datetime'],y= train['fare_amount'],s = 1, alpha= 0.2)

Vamos fazer algumas seleções para conseguir visualizar melhor esses valors entre 40 e 60 e entender quais são eles e como se distribuem

In [ ]:
fare_zoom = train['fare_amount'][(train['fare_amount']>40) & (train['fare_amount']<60)]

Temos 5993 valores entre 40 e 60 dolares 

In [ ]:
fare_zoom.shape

Com o método Counter conseguimos ver a quantidade de cada valor na faixa que estamos buscando 

In [ ]:
commom_fares_zoom = Counter(fare_zoom)

In [ ]:
commom_fares_zoom

E com o método most_commom conseguimos por exemplo visualizar os 10 valores que mais aparecem nessa faixa e a sua frequencia 


In [ ]:
most_commom_fares_zoom = commom_fares_zoom.most_common(10)

In [ ]:
most_commom_fares_zoom

Vamos plotar uma visualização da frequencia por os 10 valores mais frequentes 

In [ ]:
plt.bar([x[0] for x in most_commom_fares_zoom],[x[1] for x in most_commom_fares_zoom])

Vemos que o valor mais frequente é na faixa de 57.5 e o menos frequente é na faixa de 40 